In [ ]:
#Authors: Jason, Jacob, Austin, Grey, Alondra
#Date: 05-09-2020
#Description: Text summarizer for Amazon reviews for the Alexa product. 
#             Entire reviews are summarized into a few phrases to easily 
#             tell whether the reviewer liked the Amazon Alexa product.  


!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install -U transformers

In [ ]:
#Adding Emojis from a Database
#Alondra Lona
#Grey Todd
#Source: https://www.kaggle.com/eliasdabbas/emoji-data-descriptions-codepoints?select=emoji_df.csv
import pandas as pd
import numpy as np
from google.colab import files
import io
upload2 = files.upload()
alexadf = pd.read_csv(io.BytesIO(upload2['amazon_alexa.csv']),dtype={'verified_reviews':'str'})

Saving amazon_alexa.csv to amazon_alexa (4).csv


In [ ]:
# Jason Hubbs
#Jacob Rosner
from transformers import pipeline
from transformers.models.t5.modeling_t5 import T5Model
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead
import torch
#Code Source https://towardsdatascience.com/fine-tuning-a-t5-transformer-for-any-summarization-task-82334c64c81
#Model Source https://huggingface.co/activebus/BERT_Review
model = AutoModelWithLMHead.from_pretrained("activebus/BERT_Review") #mention specific things about the model
tokenizer = AutoTokenizer.from_pretrained("activebus/BERT_Review")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Run on GPU, if the device is available and enables to assign model training. Uses parallel processing.
model = model.to(device) #assigns model to run on that device.

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:810: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
#Source: https://towardsdatascience.com/fine-tuning-a-t5-transformer-for-any-summarization-task-82334c64c81
# Austin Escalante
# Jason Hubbs
# Jacob Rosner
data = []
x = alexadf['verified_reviews'].astype(str).values.tolist()
for i in x[0:80]: #Range can be extended to 1000. Jacob Rosner
  text = i  
  preprocess_text  = text.strip().replace("\n","")
  tokenized_text = tokenizer.encode(preprocess_text, max_length=10 ,return_tensors="pt", truncation=True).to(device)  
  split = text.split()
  length = len(split)
  if length >= 15:
    length = 15
  summary = model.generate(tokenized_text,
                        max_length = length
                        )
  data.append(tokenizer.decode(summary[0], skip_special_tokens=True))
df2 =  pd.DataFrame(data,columns = ['summary'])

Input length of input_ids is 6, but ``max_length`` is set to 3.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 5, but ``max_length`` is set to 2.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 3, but ``max_length`` is set to 1.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 4, but ``max_length`` is set to 2.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 9, but ``max_length`` is set to 3.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 10, but ``max_length`` is set to 4.This can lead to unexpected behavior. You should consider increasing ``

In [ ]:
sentiment = pipeline("sentiment-analysis")

In [ ]:
#Jacob Rosner
x = alexadf['verified_reviews'].astype(str).values.tolist()
x = np.array(x)
for i in x:
  text = i

In [ ]:
#Austin Escalante
#Jacob Rosner
data = []
for i in x[0:80]: #Range can be extended to 1000. Jacob Rosner
  text = sentiment(i)[0]
  label = text['label']
  score = round(text['score'], 2)
  if score > 0.75 and label == 'POSITIVE':
    final = "Positive 0" #Postive
  elif score > 0.75 and label == 'NEGATIVE':
    final = "Negative 1"
  elif score <= 0.75:
    final = "Neutral 2"
  else:
    final = "NA"
  data.append(final)

df4 = pd.DataFrame(data,columns = ['sentiment'])


In [ ]:
#Extractuve Word-Scoring Summarizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from transformers import BertTokenizer 

In [ ]:
#Adding Emojis from a Database
#Source: https://www.kaggle.com/eliasdabbas/emoji-data-descriptions-codepoints?select=emoji_df.csv
#Jacob Rosner
import pandas as pd
import numpy as np
from google.colab import files
import io
upload = files.upload()
df = pd.read_csv(io.BytesIO(upload['emoji_df.csv']))

Saving emoji_df.csv to emoji_df (5).csv


In [ ]:
#Emotes from https://github.com/explosion/spaCy/blob/master/spacy/lang/tokenizer_exceptions.py#L81
#@title
#Jacob Rosner
emotes = set(
    r"""
:)
:-)
:))
:-))
:)))
:-)))
(:
(-:
=)
(=
:]
:-]
[:
[-:
[=
=]
:o)
(o:
:}
:-}
8)
8-)
(-8
;)
;-)
(;
(-;
:(
:-(
:((
:-((
:(((
:-(((
):
)-:
=(
>:(
:')
:'-)
:'(
:'-(
:/
:-/
=/
=|
:|
:-|
]=
=[
:1
:P
:-P
:p
:-p
:O
:-O
:o
:-o
:0
:-0
:()
>:o
:*
:-*
:3
:-3
=3
:>
:->
:X
:-X
:x
:-x
:D
:-D
;D
;-D
=D
xD
XD
xDD
XDD
8D
8-D
^_^
^__^
^___^
>.<
>.>
<.<
._.
;_;
-_-
-__-
v.v
V.V
v_v
V_V
o_o
o_O
O_o
O_O
0_o
o_0
0_0
o.O
O.o
O.O
o.o
0.0
o.0
0.o
@_@
<3
<33
<333
</3
(^_^)
(-_-)
(._.)
(>_<)
(*_*)
(¬_¬)
ಠ_ಠ
ಠ︵ಠ
(ಠ_ಠ)
¯\(ツ)/¯
(╯°□°）╯︵┻━┻
><(((*>
""".split()
)

#tokenizer.add_tokens(":)")

In [ ]:
#Jacob Rosner
tokenizer2 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [ ]:
#Jacob Rosner
for line in emotes:
  tokenizer2.add_tokens(line)

In [ ]:
#Jacob Rosner
arr = np.array([])
for emoji in df.emoji:
  arr = np.append(arr, [emoji])

In [ ]:
#Jacob Rosner
for emoji in arr:
  tokenizer2.add_tokens(emoji)

In [ ]:
#Austin Escalante
#Jason Hubbs
#Jacob Rosner
data = []
for i in x[0:80]: #Range can be extended to 1000. Jacob Rosner
  text = i
  #tokenizer.tokenize(text)
  #Code source: https://www.youtube.com/watch?v=9PoKellNrBc
  stopwords = list(STOP_WORDS)
  tokens = tokenizer2.tokenize(text)
  punctuation = punctuation + '\n'
  word_frequencies = {}
  for word in tokens:
    if word.lower() not in stopwords:
      if word.lower() not in punctuation:
        if word not in word_frequencies.keys():
          word_frequencies[word] = 1
        else:
          word_frequencies[word] += 1
  max_freq = max(word_frequencies.values(),default = 0)
  for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_freq
  select_length = int(len(tokens)* 0.3)
  summaryExtractive = nlargest(select_length, word_frequencies, key=word_frequencies.get)
  if(len(summaryExtractive) == 0):
      summaryExtractive = text
      summaryExtractive = [text]
  if(type(summaryExtractive) == list):
    summaryList = ' '.join(summaryExtractive)
    data.append(summaryList)
  df3 =  pd.DataFrame(data,columns = ['extractive'])

In [ ]:
#Austin Escalante
#Jason Hubbs
#There are 3000+ reviews but we sliced it up to around 80 because it takes a bit of time for it all to be ran. But the range can be increased in the for loops. Comment By Jacob Rosner
frames = [alexadf, df2 ,df3, df4]
result = pd.concat([alexadf, df2, df3,df4], axis=1, join='inner')
columns_titles = [" rating", "date", "variation", "feedback", "verified_reviews", "summary", "extractive", "sentiment"]
result=result.reindex(columns=columns_titles)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', None)
result

,rating,date,variation,feedback,verified_reviews,summary,extractive,sentiment
0,5,31-Jul-18,Charcoal Fabric,1,Love my Echo!,love my echo! i,love,Positive 0
1,5,31-Jul-18,Charcoal Fabric,1,Loved it!,loved it!!,Loved it!,Positive 0
2,4,31-Jul-18,Walnut Finish,1,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you. I like being able to turn lights on and off while away from home.","sometimes while playing a game, you can accidentally or die. so",playing game answer question correctly alexa says got wrong answers like able,Neutral 2
3,5,31-Jul-18,Charcoal Fabric,1,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the lights and play games like categories. Has nice sound when playing music as well.",i have had a lot of fun with this game. i think,lot fun thing 4 y ##r old learns dinosaurs control lights,Positive 0
4,5,31-Jul-18,Charcoal Fabric,1,Music,music /,Music,Positive 0
5,5,31-Jul-18,Heather Gray Fabric,1,"I received the echo as a gift. I needed another Bluetooth or something to play music easily accessible, and found this smart speaker. Can’t wait to see what else it can do.",i received the echo as a gift. i believe that i can,received echo gift needed blue ##tooth play music easily accessible found,Positive 0
6,3,31-Jul-18,Sandstone Fabric,1,"Without having a cellphone, I cannot use many of her features. I have an iPad but do not see that of any use. It IS a great alarm. If u r almost deaf, you can hear her alarm in the bedroom from out in the living room, so that is reason enough to keep her.It is fun to ask random questions to hear her response. She does not seem to be very smartbon politics yet.","without having a cellphone, i cannot communicate. therefore,,",use alarm hear having cell ##phone features ipad great u r deaf bedroom living room reason fun ask random questions response smart ##bon politics,Neutral 2
7,5,31-Jul-18,Charcoal Fabric,1,I think this is the 5th one I've purchased. I'm working on getting one in every room of my house. I really like what features they offer specifily playing music on all Echos and controlling the lights throughout my house.,i think this is the 5th one i have. i think i,house think 5th ve purchased m working getting room like features offer spec ##if ##ily,Positive 0
8,5,30-Jul-18,Heather Gray Fabric,1,looks great,looks great!,looks great,Positive 0
9,5,30-Jul-18,Heather Gray Fabric,1,"Love it! I’ve listened to songs I haven’t heard since childhood! I get the news, weather, information! It’s great!",love it! i ’ ve listened to everything! literally!!,’ love ve listened songs haven t heard childhood,Positive 0
